### 폴더명: Asan-meningioma_Low_grade
### task
몇몇 마스크 파일이 맞지 않음.(아마도 superior와 inferior가 바뀐듯?)
확인해서 마스크 파일만 reorientation하기

### solution
jupyter로 orientation 이상한 mask를 다른 폴더에 옮기고,
mipav 메크로로 reorientation할 예정

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage
from glob import glob
import shutil

sourceDir = 'E:/Asan-meningioma_Low_grade/BrainMask'
targetDir = 'E:/Asan-meningioma_Low_grade/BrainMask'

In [ ]:
def make_dirs(path_lst):
    for path in path_lst:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass

# function that creates outputdir and return output filepath by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)

    make_dirs([os.path.join(outDir, pt)])

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath


In [ ]:
# orientation 출력하기

for (path, dirs, files) in os.walk(sourceDir):
    for file in files:
        filepath = os.path.join(path, file)
        img = nib.load(filepath)
        img_data = img.get_fdata()
        orientation = nib.aff2axcodes(img.affine)
        print(file, orientation)

우선 위와 같이 orientation을 뽑았는데 여기에는 문제 안보임. \
따라서 header를 한 번 보고 거기서도 문제 안보이면 직접 마스크 적용해서 이상한거 골라내야할 듯.

In [ ]:
# print header, affine
for (path, dirs, files) in os.walk(sourceDir):
    for file in files:
        filepath = os.path.join(path, file)
        img = nib.load(filepath)
        print(file)
        print(img.affine)


header를 print했는데 orientation 관련된 정보는 없어서 skip. \
affine을 print해도 딱히 뭐가 이상한지 모르겠음..

In [ ]:
# print header, affine
import matplotlib.pyplot as plt
%matplotlib inline

for (path, dirs, files) in os.walk(sourceDir):
    for file in files:
        filepath = os.path.join(path, file)
        img = nib.load(filepath)
        img_data = img.get_fdata()
        print(img_data.shape)
#        plt.figure()
#        plt.imshow(img_data[:, :, 50])

이번에는 register된 brain image file의 orientation을 보기로 했다.

In [ ]:
# orientation 출력하기
registerDir = "E:/Asan-meningioma_Low_grade/Registered/"

target_lst = []
for (path, dirs, files) in os.walk(registerDir):
    for file in files:
        filepath = os.path.join(path, file)
        img = nib.load(filepath)
        img_data = img.get_fdata()
        orientation = nib.aff2axcodes(img.affine)
        
        print(file, orientation)

보니까 brain image의 orientation과 skull mask의 orientation이 일치하지 않는 경우가 있음을 발견했다. \
보니까 같은 경우보다 다른 경우가 더 많아서 같은 애들을 따로 모아놓고 다른 애들만 mipav로 변경할 예정.

In [ ]:
# orientation 출력하기
registerDir = "E:/Asan-meningioma_Low_grade/Registered/"

target_lst = []
for (path, dirs, files) in os.walk(registerDir):
    for file in files:
        filepath = os.path.join(path, file)
        img = nib.load(filepath)
        img_data = img.get_fdata()
        orientation = nib.aff2axcodes(img.affine)
        if orientation[-1] == "S":
            print(file, orientation)
            target_lst.append(file)

In [ ]:
brainmask = "E:/Asan-meningioma_Low_grade/BrainMask/"
destination = "E:/Asan-meningioma_Low_grade/correct_mask/"

total = 0
for (path, dirs, files) in os.walk(brainmask):
    for file in files:
        brain_file = file.split("_")[0] + "_T1CE_registered.nii"
        if brain_file in target_lst:
            total += 1
            shutil.move(os.path.join(path, file), os.path.join(destination, file))
print(len(target_lst), total)

근데 파일을 열어보니 header 정보와 orientation이 일치하지 않는다;; \
그래서 일단 'I'인 친구들을 header 바꾸지 않고 filp 시켜서 header와 실제 이미지 orientation을 일치시키고, \
mipav에서 그 이미지를 reorientation할 예정이다.

In [ ]:
import nibabel as nib
import os
import numpy as np
from copy import deepcopy

destination = "E:/Asan-meningioma_Low_grade/BrainMask_flipped/"

for (path, dirs, files) in os.walk(brainmask):
    for file in files:
        print("Flipping {}...".format(file))
        img = nib.load(os.path.join(path, file))
        img_data = img.get_fdata()

        orientation = nib.aff2axcodes(img.affine)
        target_orientation = ('L', 'P', 'S')

        # flip
        img_flipped = np.flip(img_data, axis=2)

        # reorient된 파일을 저장하려면 아래 코드를 모두 uncomment하세요.
        ni_img = nib.Nifti1Image(img_flipped, img.affine)
        print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(ni_img.affine)))
        outPath = os.path.join(destination, file)
        nib.save(ni_img, outPath)


삐용삐용 문제 발생
task 다시 정리

1) Register에서 LPI인 애: 이미지는 그대로 두고 header만 LPS로 바꾸기->affine 바꾸기 \
2) Register에서 LPS인 애: 그대로 두기(통과) \
3) mask에서 LPI인 애: 없음(통과) \
4) maks에서 LPS인 애: 그대로 두기(통과)

In [ ]:
filepath = "E:/Asan-meningioma_Low_grade/Registered/002/002_T1CE_registered.nii"
destination = "E:/Asan-meningioma_Low_grade/"

img = nib.load(filepath)
img_data = img.get_fdata()

orientation = nib.aff2axcodes(img.affine)
target_orientation = ('L', 'P', 'S')

# flip
print(img.affine, img.affine[2])
img.affine[2] = [-i for i in img.affine[2]]
print(img.affine)
# reorient된 파일을 저장하려면 아래 코드를 모두 uncomment하세요.
ni_img = nib.Nifti1Image(img_data, img.affine)
print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(ni_img.affine)))

outPath = os.path.join(destination, "002_T1CE_registered.nii")
nib.save(ni_img, outPath)


affine의 z축에 -1을 곱하면 orientation이 바뀐다는걸 알았다! -> 이거 아님!!!! reorientation_final 참고할 것.
이제 LPI인 register이미지에 적용하자.

In [ ]:
registerDir = "E:/Asan-meningioma_Low_grade/Registered/"
destination = "E:/Asan-meningioma_Low_grade/Registered_corrected"
total = 0

for (path, dirs, files) in os.walk(registerDir):
    for file in files:
        total += 1
        pt = path.split('/')[-1]
        img = nib.load(os.path.join(path, file))
        img_data = img.get_fdata()
        orientation = nib.aff2axcodes(img.affine)
        
        outPath = get_outpath(destination, pt, file)
        if outPath is None:
            continue

        if 'S' in orientation:
            print("#{} Copying {}...".format(total, file))
            shutil.copyfile(os.path.join(path, file), outPath)
            continue
        print("#{} Reorient {}...".format(total, file))

        # flip
        img.affine[2] = [-i for i in img.affine[2]]
        ni_img = nib.Nifti1Image(img_data, img.affine)
        print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(ni_img.affine)))
        nib.save(ni_img, outPath)


09/10 

교수님께서 orientation이 틀려도 좋으니 register 파일은 건드리지 말아달라고 하셨다. \
skull mask만 건드릴 예정. mipav로 reorient 해보고 affine 바뀐 걸 그대로 적용할 예정이다 \
(reorientation_final에서 사용한 방법과 동일.)


In [ ]:
registerDir = "E:/Asan-meningioma_Low_grade/Registered/"
maskDir = 'E:/Asan-meningioma_Low_grade/BrainMask'
targetDir = 'E:/Asan-meningioma_Low_grade/BrainMask_corrected'
total = 0

for (path, dirs, files) in os.walk(registerDir):
    for file in files:
        total += 1
        pt = path.split('/')[-1]
        
        img = nib.load(os.path.join(path, file))
        img_data = img.get_fdata()
        orientation = nib.aff2axcodes(img.affine)

        maskfile = os.path.join(maskDir, pt+"_brain_mask.nii.gz")
        outPath = os.path.join(targetDir, pt+"_brain_mask.nii.gz")
        if os.path.exists(outPath):
            continue

        if 'S' in orientation:
            print("#{} Copying {}...".format(total, pt+"_brain_mask.nii.gz"))
            shutil.copyfile(maskfile, outPath)
            continue
#        print("#{} Reorient {}...".format(total, file))

        # flip
#        img.affine[2] = [-i for i in img.affine[2]]
#        ni_img = nib.Nifti1Image(img_data, img.affine)
#        print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(ni_img.affine)))
#        nib.save(ni_img, outPath)


시도 1: image를 flip 하지 않고 affine만 flip (LPS->LPI) \
시도 2: image를 flip 하고 affine을 flip 안함 (LPS->LPS) \
시도 3: image를 flip 하고 affine도 flip 함 (LPS->LPI)


시도 1, 2 모두 성공. But 시도2는 mask 적용하면 LPI로 나오고 시도2는 mask 적용하면 LPS로 나옴. \
=> 시도1로 함. 만들어진 파일은 header(affine)은 LPI이고 실제는 image LPS임.mask apply도 정상적으로 됨.

In [ ]:
# 파일 1개 demo
filepath = "E:/Asan-meningioma_Low_grade/BrainMask/002_brain_mask.nii.gz"
path = "E:/Asan-meningioma_Low_grade/BrainMask_corrected/"
file = "002_brain_mask.nii.gz"

img = nib.load(filepath)

img_data = img.get_fdata()
affine = np.empty_like(img.affine)
affine[:] = img.affine

orientation = nib.aff2axcodes(img.affine)
target_orientation = ('L', 'P', 'S')
print(img.affine)

# filp
#img_flipped = np.flip(img_data, axis=2)
affine[:,2]*=-1
if affine[2,-1] > 0:
    affine[2,-1] = affine[2,-1] - img_data.shape[1] + 1

elif affine[2,-1] < 0:
    affine[2,-1] = affine[2,-1] + img_data.shape[1] - 1
    
print(affine)
print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(affine)))
# reorient된 파일을 저장하려면 아래 코드를 모두 uncomment하세요.
ni_img = nib.Nifti1Image(img_data, affine)
outPath = os.path.join(path, file)
nib.save(ni_img, outPath)
print(ni_img.affine)

In [ ]:
# mask 적용해보기 -> OK
srcpath = "E:/Asan-meningioma_Low_grade/Registered/002"
targetpath = "E:/Asan-meningioma_Low_grade/"
maskpath = "E:/Asan-meningioma_Low_grade/BrainMask_corrected/002_brain_mask.nii.gz"

for (path, dirs, files) in os.walk(srcpath):
    for file in files:
        nib_file = nib.load(os.path.join(path, file))
        mask = nib.load(os.path.join(maskpath))

        masked_data = np.multiply(nib_file.get_data(), mask.get_data())
        masked_data = nib.Nifti1Image(masked_data, nib_file.affine, nib_file.header)
        if not os.path.exists(os.path.join(targetpath)):
            os.mkdir(os.path.join(targetpath))
        outPath = os.path.join(targetpath, file)
        nib.save(masked_data, outPath)


In [ ]:
# 파일 전체
Srcpath = "E:/Asan-meningioma_Low_grade/BrainMask/"
targetDir = "E:/Asan-meningioma_Low_grade/BrainMask_corrected/"

for (path, dirs, files) in os.walk(Srcpath):
    for file in files:
        outPath = os.path.join(targetDir, file)
        if os.path.exists(outPath):
            print("{} already exists".format(file))
            continue

        filepath = os.path.join(path, file)
        img = nib.load(filepath)

        img_data = img.get_fdata()
        affine = np.empty_like(img.affine)
        affine[:] = img.affine

        orientation = nib.aff2axcodes(img.affine)
        target_orientation = ('L', 'P', 'S')

        # filp
        affine[:,2]*=-1
        if affine[2,-1] > 0:
            affine[2,-1] = affine[2,-1] - img_data.shape[1] + 1

        elif affine[2,-1] < 0:
            affine[2,-1] = affine[2,-1] + img_data.shape[1] - 1

        print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(affine)))
        # reorient된 파일을 저장하려면 아래 코드를 모두 uncomment하세요.
        ni_img = nib.Nifti1Image(img_data, affine)
        nib.save(ni_img, outPath)